This notebook is for descriptive analysis purposes of the dataset

In [40]:
import kagglehub
import torch
import torch.nn as nn
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader, random_split
import torchvision.models as models
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
from torchvision.models import resnet18, ResNet18_Weights
import numpy as np

path = kagglehub.dataset_download("jessicali9530/caltech256")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# There are 256 object categories
num_classes = 256
num_epochs = 10
learning_rate = 0.001

# Transformations for the images
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


In [41]:
# Loading the dataset
dataset = ImageFolder(root = path + "/256_ObjectCategories", transform=transform)

# Splitting into train and test sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# CNN to extract image features
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        # Processes the image. Outputs feature vector
        resnet = models.resnet18(weights=ResNet18_Weights.DEFAULT)
        # Removes last classification layer
        modules = list(resnet.children())[:-1]
        self.resnet = nn.Sequential(*modules)
    
    def forward(self, x):
        with torch.no_grad():
            features = self.resnet(x)
            # Flatten then return
            return features.view(features.size(0), -1)

In [44]:
# They are not the same. This is not the problem
train_loader == test_loader

False

In [ ]:
model = CNN()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
n_total_steps = len(train_loader)

# sklearn models
random_forest = RandomForestClassifier()
decision_tree = DecisionTreeClassifier()
svc = SVC()

train_outputs = []
train_labels = []
# Looping through CNN and getting the outputs then
# Removing gradient tracking, convert to numpy, then
# Append to regular arrays to later fit into ML models
for images, labels in train_loader:
    output = model(images)
    train_outputs.append(output.detach().numpy())
    train_labels.append(labels.detach().numpy())
    # All the labels are the same
    
X = np.vstack(train_outputs)
y = np.hstack(train_labels)

random_forest.fit(X, y)
decision_tree.fit(X, y)
svc.fit(X, y) 

,criterion,'gini'
,splitter,'best'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,None
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,None


In [46]:

print(np.unique(y))

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 243 244 245 24

In [ ]:
test_outputs = []
test_labels = []

for images, labels in test_loader:
    output = model(images)
    test_outputs.append(output.numpy())
    test_labels.append(labels.numpy())

X = np.vstack(test_outputs)
y = np.hstack(test_labels)

r_pred = random_forest.predict(X)
d_pred = decision_tree.predict(X)
s_pred = svc.predict(X) 

print(accuracy_score(y, r_pred))
print(accuracy_score(y, d_pred))
print(accuracy_score(y, s_pred))

0.5059217512047701
0.2468349260802091
